In [3]:
# This is the script that compiles the original Best Track Data files from CMA.
# Input files: /home/lzhou/Precipitation/TC_Tracks/CMA_Historical_Data/Track_Data/CMABSTdata
# Output directory: /home/lzhou/Precipitation/Precipitation_Scripts/Output
# Output files: CMA_Best_Tracks.csv, CMA_Best_Tracks.shp and CMA_Best_Tracks_Nodes.shp

In [4]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns

from shapely.geometry import LineString, Point, Polygon
import geopandas as gpd
from pyproj import CRS
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
#out_dir = '/home/lzhou/Precipitation/Precipitation_Scripts/Output'
out_dir = r'D:\GitHub\Precipitation_Scripts\Output'

In [6]:
#track_folder = '/home/lzhou/Precipitation/TC_Tracks/CMA_Historical_Data/Track_Data/CMABSTdata'
track_folder = r'D:\Precipitation\CMA_Historical_Data\Track_Data\CMABSTdata'
files = os.listdir(track_folder)
files = [x for x in files if 'txt' in x]

In [7]:
first_read = 1
ar = 0
for year in np.arange(1949,2021):
    print(year)
    filename = 'CH'+str(year)+'BST.txt'
    infile = os.path.join(track_folder,filename)
    os.path.isfile(infile)

    f = open(infile)
    content = f.readlines()
    f.close()
    content = [x.split() for x in content]

    fr = 0
    while fr < len(content):
        #print(content[fr])
        ss = content[fr]
        nr = int(ss[2])    #number of rows for this track 
        nsk = fr + 1       #rows that the record starts
        
        #print('rows to skip, rows to read: ', nsk, nr)
    
        # read in the track record. 
        # column unit: Time(UTC), IntensityCategory(Chinese standard), LAT/LON(degree in 0.1),
        # PRES(hPa), WND(2-min mean max. sustained wind near the TC center m/s)
        try:
            dummy = pd.DataFrame(content[nsk:nsk+nr], \
                              columns=['Time','IntensityCategory','LAT','LON','PRES','WND'])
        except:
            dummy = pd.DataFrame(content[nsk:nsk+nr], \
                              columns=['Time','IntensityCategory','LAT','LON','PRES','WND','OWD'])
        
        if first_read == 1:
            first_read = 0
            aa = dummy.copy()
        else:
            aa = aa.append(dummy,ignore_index=True)
            
        aa.loc[ar:ar+nr,'IntID'] = ss[1]
        aa.loc[ar:ar+nr,'InYearID'] = ss[3]
        aa.loc[ar:ar+nr,'CNID'] = ss[4]
        aa.loc[ar:ar+nr,'Flag'] = ss[5]            # Flag of the last data line: 0=Decay, 1=Move outside of the region, 2=Merge, 3 = Quais-Stationary
        aa.loc[ar:ar+nr,'TimeInterval'] = ss[6]    # Time interval (hour)
        aa.loc[ar:ar+nr,'Name'] = ss[7]            # Typhoon Name
        aa.loc[ar:ar+nr,'CMAID'] = year*100+int(ss[3])
            
        ar = ar + nr
        fr = fr + nr + 1 

# convert data types
#aa.IntID = aa.IntID.astype('int')
aa.InYearID = aa.InYearID.astype('int')
#aa.CNID = aa.CNID.astype('int')
aa.Flag = aa.Flag.astype('int')
aa.TimeInterval = aa.TimeInterval.astype('int')
aa.CMAID = aa.CMAID.astype('int')

# correc lat and lon
aa.LAT = aa.LAT.astype('float')
aa.LON = aa.LON.astype('float')
aa.LAT = aa.LAT/10.
aa.LON = aa.LON/10.

# convert integers to datetime format and get year and month info
aa['Time'] = pd.to_datetime(aa['Time'],format='%Y%m%d%H')
aa['Year'] = pd.DatetimeIndex(aa['Time']).year
aa['Month'] = pd.DatetimeIndex(aa['Time']).month

1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [8]:
cma_tracks = aa.copy()
#outfile = os.path.join(out_dir,'CMA_Best_Tracks.csv')
#cma_tracks.to_csv(outfile,index=False)

In [9]:
# load the best track csv file
#infile = 'CMA_Best_Tracks.csv'
#df= pd.read_csv(infile)
df = cma_tracks.copy()
#df=df[df.Year==2019]

# choose initialization projection
init_epsg = 4326    # WGS 1984

# convert to geopandas dataframe
df['geometry'] = df.apply(lambda x: Point((float(x.LON),float(x.LAT))),axis=1)
df = gpd.GeoDataFrame(df,geometry='geometry')
df.crs = CRS.from_epsg(init_epsg)

In [10]:
# connect nodes to tracks based on CMAID
df2 = df.groupby(['CMAID'])['geometry'].apply(lambda x: LineString(x.tolist()))
df2 = gpd.GeoDataFrame(df2,geometry='geometry')

# get the information of each track based on CMAID
df3 = df[['CMAID','Year','IntID','InYearID','CNID','Flag','Name']].copy()
df3 = df3.drop_duplicates()
df3.set_index('CMAID',inplace=True)

# merge the compiled tracks and tracks' info
tracks = df3.merge(df2,left_index=True,right_index=True)
tracks = gpd.GeoDataFrame(tracks,geometry='geometry')
tracks.crs = CRS.from_epsg(init_epsg)

In [12]:
# save to shapefiles
tracks_nodes = df.copy()

tracks_nodes['Day'] = pd.DatetimeIndex(tracks_nodes['Time']).day
tracks_nodes['Hour'] = pd.DatetimeIndex(tracks_nodes['Time']).hour

tracks_nodes = tracks_nodes[['Year','Month','Day','Hour','CMAID','InYearID','IntID','CNID','Name', \
                             'Flag','TimeInterval','IntensityCategory','LAT','LON','PRES','WND','OWD','geometry']]

outfile1 = os.path.join(out_dir,'CMA_Best_Tracks_Nodes.shp')
tracks_nodes.to_file(outfile1)

outfile2 = os.path.join(out_dir,'CMA_Best_Tracks.shp')
tracks.to_file(outfile2)



E:\Users\zhouluxi\miniconda3\envs\netcdf\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  # This is added back by InteractiveShellApp.init_path()
